In [1]:
#basics -> imports and apikey
import json
import requests as rq
import pandas as pd
import time

apikey = 'YOUR API KEY'

### I'm just using an excel table where I have the list of summoner names

In [ ]:
list_summoners = pd.read_excel('LVPSummoner.xlsx',
                      sheetname = 'table_output',
                      header = 0,
                      index_col = 0,
                      parse_cols = "A, B",
                      convert_float = True)

#check what was actually extracted into the table
print list_summoners.head()
print list_summoners.count()

### I use PyMongo (for storing into MongoDB), but if you don't want to store you can skip anything DB related.

In [4]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

In [5]:
db = client.lol_stats_db
collection = db.summoner_stats

### Old code, as you can see. API version 2.2 still ;)

In [7]:
#test with i=0
r = rq.get('https://euw.api.pvp.net/api/lol/euw/v1.3/stats/by-summoner/' +str(list_summoners.index[0])+
           '/summary?season=SEASON2016&api_key=' + apikey)
#check response code 
r.status_code

200

In [9]:
(r.status_code == 200)

True

### Insert into database

### Id=0 for testing

In [ ]:
#insert into database
###result = db.match_list_2016_5x5.remove({ "_id": str(list_summoners.index[0])})
result = db.summoner_stats.insert_one({ "_id": str(list_summoners.index[0]), "stats": json.loads(r.text)} )
print result.inserted_id

### Retrieve stats (for example)

In [11]:
#let's get all stats for each summoner in SEASON 2016
i = 1
while i < len(list_summoners.index):  
    ##result = db.match_list_2016_5x5.remove({ "_id": str(list_summoners.index[i])})
    #get all matches for a summoner id
    #WARNING: let's do only one request every two seconds
    r = rq.get('https://euw.api.pvp.net/api/lol/euw/v1.3/stats/by-summoner/' +str(list_summoners.index[i])+
           '/summary?season=SEASON2016&api_key=' + apikey)
    #check response code
    while (r.status_code <> 200):
        time.sleep(4)
        r = rq.get('https://euw.api.pvp.net/api/lol/euw/v1.3/stats/by-summoner/' +str(list_summoners.index[i])+
           '/summary?season=SEASON2016&api_key=' + apikey)
    result = db.summoner_stats.insert_one({ "_id": str(list_summoners.index[i]), "stats": json.loads(r.text)} )
    #insert into stats (contains all stats per summoner in season 16)
    #the summoner ID as DB Key    
    #increase and wait    
    i = i + 1
    time.sleep(1)

### Each API response was stored into a MongoDB above; below I just retrieve an individual instance and then, using the JSON structure, I test a few variables.

In [27]:
cursor = db.summoner_stats.find({"_id": str(list_summoners.index[0])})
for document in cursor:
    print document['stats']['playerStatSummaries'][0]['playerStatSummaryType']  == 'AramUnranked5x5'
    print document['stats']['playerStatSummaries'][0]['aggregatedStats']['totalChampionKills']
    
if (document['stats']['playerStatSummaries'][0]['aggregatedStats']):
    print "OK"

True
6022
OK


### Let's extract all (wanted) stats per summoner. This is using the JSON from the Mongo Database, but you could feed the API response directly. But, honestly, it is better to have a middle DB; this way you can run the query (it can take long) and do the analysis much later without worrying.

In [28]:
#we want wins from 'RankedFlexSR', 'Unranked'
#wins and losses from 'RankedSolo5x5', 'RankedTeam5x5'
#and total wins per summoner


i=0
summoner_list = []
total_wins = []
total_ranked_wins = []
total_ranked_losses = []
total_ranked_kills = []
total_ranked_assists = []

while (i<len(list_summoners)):
    cursor = db.summoner_stats.find({"_id": str(list_summoners.index[i])})
    for document in cursor:
        summoner_list.append(document['_id'])
        wins = 0
        ranked_wins = 0
        ranked_losses = 0
        ranked_kills = 0
        ranked_assists = 0
        j=0
        while (j<len(document['stats']['playerStatSummaries'])):
            queue = document['stats']['playerStatSummaries'][j]['playerStatSummaryType']
            if (queue == 'RankedFlexSR' or queue == 'Unranked'):
                wins = wins + document['stats']['playerStatSummaries'][j]['wins']
            elif (queue == 'RankedSolo5x5' or queue == 'RankedTeam5x5'):
                wins = wins + document['stats']['playerStatSummaries'][j]['wins']
                ranked_wins = ranked_wins + document['stats']['playerStatSummaries'][j]['wins']
                ranked_losses = ranked_losses + document['stats']['playerStatSummaries'][j]['losses']
                #check whether list is empty
                if (document['stats']['playerStatSummaries'][j]['aggregatedStats']):
                    ranked_kills = ranked_kills + document['stats']['playerStatSummaries'][j]['aggregatedStats']['totalChampionKills']
                    ranked_assists = ranked_assists + document['stats']['playerStatSummaries'][j]['aggregatedStats']['totalAssists']
                else:
                    ranked_kills = ranked_kills + 0
                    ranked_assists = ranked_assists + 0
            j=j+1
        total_wins.append(wins)
        total_ranked_wins.append(ranked_wins)
        total_ranked_losses.append(ranked_losses)
        total_ranked_kills.append(ranked_kills)
        total_ranked_assists.append(ranked_assists) 
    i=i+1
        
print len(summoner_list)
print len(total_wins)
print len(total_ranked_assists)

440
440
440


### Above I only added all the "wanted" variables to five different lists. From these ordered lists (and the ID) I'll build a pandas dataframe easily:

In [30]:
d = {'SummonerID' : summoner_list,'totalWins' : total_wins, 'totalRankedWins' : total_ranked_wins,
     'totalRankedLosses' : total_ranked_losses, 'totalRankedKills' : total_ranked_kills,
     'totalRankedAssists' : total_ranked_assists}
table_summoner_stats = pd.DataFrame(d)

### An then you can save it to any desired format; here, it was excel, but CSV should be even simpler.

In [32]:
#Now, let's move this to an excel file

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('SummonerStats.xlsx', engine='xlsxwriter')
# Convert the dataframe to an XlsxWriter Excel object.
table_summoner_stats.to_excel(writer, sheet_name='table_summoner_stats')
# Close the Pandas Excel writer and output the Excel file.
writer.save()